In [1]:
clean_up = True
%run stdPackages.ipynb

Load all types of nesting trees from:

In [2]:
FunnyName_NS = {n+'_input': n for n in ('Y1','Y2')}
data_str = os.path.join(d['data'], 'Nestings.xlsx')
read_trees = {'CES1': {'CES1': {'f': 'CES'}},
              'CES2': {'CES2': {'f':'CES'}},
              'CES1_norm': {'CES1': {'f':'CES_norm'}}, 
              'CES2_norm': {'CES2': {'f':'CES_norm'}},
              'CET1': {'CET1': {'f': 'CET'}},
              'CET2': {'CET2': {'f': 'CET'}},
              'CET1_norm': {'CET1': {'f': 'CET_norm'}},
              'CET2_norm': {'CET2': {'f': 'CET_norm'}},
              'CES_CET': {'CES1': {'f': 'CES'}, 'CET1': {'f': 'CET'}},
              'CES_CET_norm': {'CES1': {'f':'CES'}, 'CET1': {'f':'CET_norm'}},
              'CESCET_norm': {'CES1': {'f':'CES_norm'}, 'CET1': {'f': 'CET_norm'}},
              'CESCET3': {'CES1': {'f':'CES'}, 'CET3': {'f':'CET'}},
              'CESCET3_norm': {'CES1': {'f':'CES_norm'}, 'CET3': {'f':'CET_norm'}},
              'FunkyTree': {'FunkyTree_CET': {'f':'CET_norm'}, 'FunkyTree_CES': {'f':'CES_norm'}},
              'ExtremeFunk': {'ExtremeF1': {'f': 'CES_norm'}, 'ExtremeF2': {'f':'CET_norm'}},
              'FunnyName': {'FunnyNameInp': {'f':'CES_norm'}, 'FunnyNameOut': {'f':'CET_norm'}}}
Trees = {k: nestingTree.aggTree_from_data(data_str, read_trees = read_trees[k], name = k)(namespace=FunnyName_NS) for k in read_trees} # init b

Load global settings:

In [3]:
glob = CGE_globals.SmallOpen(kwargs_vals = {'t': range(1,3)})

Initialize production module for each:

In [4]:
ws = gams.GamsWorkspace(working_directory=d['work'])
Ps = {k: CGE_Production.Production(tree = Trees[k], glob = glob, ns = {}, s_kwargs = {'ws': ws}) for k in Trees}

#### Test baseline mode

*Init states:*

In [5]:
[P.compile(initDB=True) for P in Ps.values()];

*Write text:*

In [6]:
[P.write() for P in Ps.values()];

*Run models (with same workspace):*

In [7]:
Ms = {k: Ps[k].run(exportTo=d['work'], ws=ws,**{'cns': 'CONOPT4'}) for k in Ps}

#### Test calibration mode:

*Change state:*

In [8]:
[setattr(p.s,'state','C') for p in Ps.values()];

*Update database to baseline solution:*

In [9]:
[setattr(Ps[k].s,'db',Ms[k].out_db) for k in Ps];

*Write:*

In [10]:
[P.write() for P in Ps.values()];

*Re-run:*

In [ ]:
Ms = {k: Ps[k].run(exportTo=d['work'], ws=ws,**{'cns': 'CONOPT4'}) for k in Ps}

## Test version 2

Load all types of nesting trees from:

In [2]:
FunnyName_NS = {n+'_input': n for n in ('Y1','Y2')}
data_str = os.path.join(d['data'], 'Nestings.xlsx')
read_trees = {'CES1': {'CES1': {'f': 'CES'}},
              'CES2': {'CES2': {'f':'CES'}},
              'CES1_norm': {'CES1': {'f':'CES_norm'}}, 
              'CES2_norm': {'CES2': {'f':'CES_norm'}},
              'CET1': {'CET1': {'f': 'CET'}},
              'CET2': {'CET2': {'f': 'CET'}},
              'CET1_norm': {'CET1': {'f': 'CET_norm'}},
              'CET2_norm': {'CET2': {'f': 'CET_norm'}},
              'CES_CET': {'CES1': {'f': 'CES'}, 'CET1': {'f': 'CET'}},
              'CES_CET_norm': {'CES1': {'f':'CES'}, 'CET1': {'f':'CET_norm'}},
              'CESCET_norm': {'CES1': {'f':'CES_norm'}, 'CET1': {'f': 'CET_norm'}},
              'CESCET3': {'CES1': {'f':'CES'}, 'CET3': {'f':'CET'}},
              'CESCET3_norm': {'CES1': {'f':'CES_norm'}, 'CET3': {'f':'CET_norm'}},
              'FunkyTree': {'FunkyTree_CET': {'f':'CET_norm'}, 'FunkyTree_CES': {'f':'CES_norm'}},
              'ExtremeFunk': {'ExtremeF1': {'f': 'CES_norm'}, 'ExtremeF2': {'f':'CET_norm'}},
              'FunnyName': {'FunnyNameInp': {'f':'CES_norm'}, 'FunnyNameOut': {'f':'CET_norm'}}}
Trees = {k: nestingTree.aggTree_from_data(data_str, read_trees = read_trees[k], name = k)(namespace=FunnyName_NS) for k in read_trees} # init b

Load global settings:

In [3]:
glob = CGE_globals.SmallOpen(kwargs_vals = {'t': range(1,3)})

Initialize production module for each:

In [4]:
ws = gams.GamsWorkspace(working_directory=d['work'])
Ps = {k: CGE_Production.Production_ExoMu(tree = Trees[k], glob = glob, ns = {}, s_kwargs = {'ws': ws}) for k in Trees}

In this second version, we only endogenize share parameters for variables in the IO data and not for intermediate goods. We can still run into some problems with linear dependence if we include scale preserving nests. We handle this as follows:
* For all inputs, identify a single element in $\mu[s,n,nn]$ to be endogenized. This fits with the corresponding number of demand components that are fixed $qD$.
* If an entire input-type nest is scale preserving, this, however, will not work. So, we pick out a single element $qD$ to keep endogenous and corresponding $mu$ to be exogenous. This is done as follows:
    * Remove all ```endo_mu``` combinations from ```map_spinp```. If a combination (s,n) is entirely removed, this means that there is an issue.
    * If there is

If a sector has more than one output, we similarly have to adjust some share parameters in the calibration mode. Specifically, because we have price equations on the form $p[s,n] = (1+markup[s])(pS[s,n]+tauS[s,n]+ic[s,n])$, where all the prices $p[s,n]$ are fixed - and we only have one markup to adjust - we need technology parameters to ensure that the corret price levels $pS[s,n]$ can be acieved. This is done by unfixing one share parameter for each output from a sector except for one. Specifically we do the following:
* Identify all share parameters where $n$ is an output.
* Remove all elements in ```endo_mu``` to make sure that we are not unfixing an already endogenous parameter.
* Select one share parameter for all outputs except one.

#### Test baseline mode

*Init states:*

In [5]:
[P.compile(initDB=True) for P in Ps.values()];

*Write text:*

In [6]:
[P.write() for P in Ps.values()];

*Run models (with same workspace):*

In [7]:
Ms = {k: Ps[k].run(exportTo=d['work'], ws=ws,**{'cns': 'CONOPT4'}) for k in Ps}

*Change state:*

In [8]:
[setattr(p.s,'state','C') for p in Ps.values()];

*Update database to baseline solution:*

In [9]:
[setattr(Ps[k].s,'db',Ms[k].out_db) for k in Ps];

*Write:*

In [10]:
[P.write() for P in Ps.values()];

*Re-run:*

### Calibration subsets:

From an inputs side, suggest all the $\mu$'s that could be endogenized:
1. All the elements in ```map``` where [s,nn] are inputs can be endogenized. However, only one combination (s,n,nn) should be endogenized.
2. Next, we identify the (s,n) combinations that can be trouble: These are the scale preserving input nests, where all branches are inputs. We can identify them by: 
     1. selecting the entire ```map_spinp``` mapping, 
     2. extracting the endogenous elements, and 
     3. checking if any (s,n) combinations are left out.
3. Next, for sectors with multiple outputs, we need to adjust share parameters on the output side as well. Specifically, for each sector, we need to endogenize i-1 share parameters where i is the number of outputs from the sector. 
4. Given the three, we need to choose:
    * For each (s, nn) in ```endo_mu_all``` suggest one combination (s,n,nn) that is to be endogenous. Call this ```endo_mu```. If possible, this combination should not include elements from ```out_mu```, because these might be needed to ensure that ```endo_mu_out``` can be selected. However, for some tree types, this has to be done (see the tree-type ```CET1``` below).
    * For each (s,n) nest that is identified by ```spinp_Trouble```, choose one element (s,n,nn) to be exogenous. If this element was in ```endo_mu```, make sure that the corresponding ```qD[s,nn]``` is included in ```endo_qD```.
    * For each sector that produces multiple outputs, suggest $i-1$ share parameters to endogenize. Make sure that these have not been chosen for the other two subset of $\mu$'s.
    
Note that this method does guarantee that a solution - in particular if the **entire** nest is scale preserving. The trees ```FunkyTree``` and ```ExtremeFunk``` are examples of this below: There is actually no way to identify share parameters to make these nesting structure work with the current formulation; thus, when running these in calibration mode, they solve to a singular solution.

In [12]:
p = Ps['FunkyTree']
self = p

*1. All elements that can be endogenized:*

In [13]:
endo_mu_all = gpyDB_wheels.adj.rc_pd(self.get('map'), self.get('input').rename(['s','nn']))

*2. All nests that may be trouble:*

In [14]:
spinp_Trouble =gpyDB_wheels.adj.rc_pd(self.get('map_spinp'), ('not', self.get('map_spinp').difference(endo_mu_all).droplevel('nn').unique()))

*3. All potentially endogenizeable share parameters on the output side - the ones with multiple outputs:*

In [15]:
mu_out = gpyDB_wheels.adj.rc_pd(self.get('map'), self.get('output'))
s_numberOut = mu_out.to_frame(index=False).groupby('s').nunique()['n']
mout_s =s_numberOut[s_numberOut>1].index
mu_out = gpyDB_wheels.adj.rc_pd(mu_out, mout_s)

*4. Suggest combination of $\mu$:*

In [16]:
endo_mu = self.uniqueFromMap(endo_mu_all.difference(mu_out), gb = ['s','nn']) # suggest endogenous combination that does not include elements  from mu_out
x = self.uniqueFromMap(endo_mu_all, gb = ['s','nn']) # suggest endogenous combination from all endo_mu_all
endo_mu = endo_mu.union(gpyDB_wheels.adj.rc_pd(x, ('not', endo_mu.droplevel('n')))) # if there is a combination of (s,nn) in x that is missing, add it here. 
exo_mu  = self.uniqueFromMap(spinp_Trouble, gb = ['s','n']) # suggest making this element exogenous
exo_mu_out = self.uniqueFromMap(mu_out.intersection(endo_mu), gb ='s') # suggest NOT making this endogenous. 
y = self.uniqueFromMap(mu_out, gb = 's') # suggest not making this endogenous either - selecting one for each
exo_mu_out = exo_mu_out.union(gpyDB_wheels.adj.rc_pd(y, ('not', exo_mu_out.droplevel('nn')))) # if there is a combination of (s,n) in y that is missing, add it here.
endo_mu_out = self.uniqueFromMap(gpyDB_wheels.adj.rc_pd(mu_out, ('not', exo_mu_out.droplevel('nn'))), gb = ['s','n'])

*5. Check that combination is successfull, i.e. we are able to endogenize the right number of share parameters:*

In [17]:
len(endo_mu_out) == sum(mu_out.to_frame(index=False).groupby('s').nunique()['n']-1)

True

In [ ]:
endo_mu_out.union(endo_mu).difference(exo_mu)

MultiIndex([('s1', 'X1', 'Z1'),
            ('s1', 'X2', 'Z2'),
            ('s1', 'Y1', 'Z1'),
            ('s1', 'Y2', 'Z2')],
           names=['s', 'n', 'nn'])

In [25]:
Ms = {k: Ps[k].run(exportTo=d['work'], ws=ws,**{'cns': 'CONOPT4'}) for k in Ps}